# Astrobrowser - Herschel Reference Survey

Explore the HiPS maps available for the galaxies in the HRS and compute aperture photometry.

# 1. Initialisation

In [1]:
%matplotlib ipympl
import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.table import Table
from astropy import units as u
from scripts import astrobrowser
from photutils.aperture import SkyEllipticalAperture
import importlib

## Read input catalogue

In [2]:
HRS_catalogue = Table.read('HRS/cortese_2014_table2.vot')

In [3]:
HRS_catalogue

HRS,CGCG,VCC,UGC,NGC,IC,R.A.__J.2000_,Dec__J.2000_,Type,FLAG_100,F_100,sigma_100,FLAG_160,F_160,sigma_160,a,b,P.A.,Proposal_ID
,,,,,,hms,dms,,,Jy,Jy,,Jy,Jy,arcsec,arcsec,deg,
int16,str7,int16,int16,int16,int16,str11,str11,int16,int16,float32,float32,int16,float32,float32,float32,float32,float32,str29
1,123-035,0,0,0,0,10:17:39.66,+22:48:35.9,13,1,0.748,0.169,1,0.932,0.079,48.0,40.0,-10.0,OT1_lcortese_1
2,124-004,0,5588,0,0,10:20:57.13,+25:21:53.4,5,1,2.439,0.227,1,2.808,0.179,47.0,45.0,40.0,OT1_lcortese_1
3,94-026,0,5617,3226,0,10:23:27.01,+19:53:54.7,0,0,0.0,0.0,2,0.846,0.087,39.0,39.0,15.0,GT1_lspinogl_2
4,94-028,0,5620,3227,0,10:23:30.58,+19:51:54.2,3,2,17.589,1.104,2,22.675,1.165,100.0,84.0,-25.0,GT1_lspinogl_2/OT2_aalonsoh_2
5,94-052,0,0,0,610,10:26:28.37,+20:13:41.5,7,1,4.502,0.331,1,5.563,0.528,81.0,48.0,28.0,OT1_lcortese_1
6,154-016,0,5662,0,0,10:27:01.16,+28:38:21.9,5,1,0.275,0.14,1,0.483,0.085,62.0,22.0,-30.0,OT1_lcortese_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,47-070,0,9328,5645,0,14:30:39.35,+07:16:30.3,9,1,4.998,0.406,1,6.331,0.483,101.0,63.0,-75.0,OT1_lcortese_1


# 2. Aperture photometry

Consider only the first $n$ galaxies (for debugging)

In [4]:
n = len(HRS_catalogue)
n = 10
print(f'Considering the first {n} galaxies')

Considering the first 10 galaxies


In [5]:
def find_bg(data):
    p16, p50 = np.nanpercentile(data, [16, 50])
    mu0 = p50
    sigma0 = p50 - p16
    weight = np.exp(-.5 * ((data - mu0) / sigma0)**2)
    total_weight = np.nansum(weight)
    mu1 = np.nansum(weight * data) / total_weight
    sigma1 = np.nansum(weight * data**2) / total_weight
    sigma1 = np.sqrt(sigma1 - mu1**2)
    print(mu0, sigma0)
    print(mu1, sigma1)
    #ivar = 1/sigma1**2 - 1/sigma0**2
    #mu = (mu1/sigma1**2 - mu0/sigma0**2) / ivar
    #print(mu, np.sqrt(1/ivar))
    #return mu, np.sqrt(1/ivar)
    return mu1, sigma1
    

importlib.reload(astrobrowser)
band = 'PACS160'
hips_service_url = 'http://skies.esac.esa.int/Herschel/PACS160'
beam = (2.85 * u.arcsec)**2
skymap_units = u.Jy / beam

pixel_arcsec = 2.
pixel_area = (pixel_arcsec * u.arcsec)**2  # cos(DEC) due to Mercator projection added below
unit_conversion = (skymap_units * pixel_area).to_value(u.Jy)

with PdfPages(os.path.join('HRS', f'{band}_maps.pdf')) as pdf:
    flux_Jy = []
    flux_err_Jy = []
    for galaxy in HRS_catalogue[:n]:
    
        position = SkyCoord(galaxy['R.A.__J.2000_'], galaxy['Dec__J.2000_'],  unit=(u.hourangle, u.deg))
        radius_arcsec = 2 * galaxy['a']
        print(f"> Downloading HRS-{galaxy['HRS']}:")
        header, data = astrobrowser.get_cutout(hips_service_url, position.ra.deg, position.dec.deg, radius_arcsec, pixel_arcsec)
        if header is None:
            flux_Jy.append(np.nan)
            flux_err_Jy.append(np.nan)
            continue
        wcs = WCS(header)

        aperture = SkyEllipticalAperture(position, a=galaxy['a']*u.arcsec, b=galaxy['b']*u.arcsec, theta=-galaxy['P.A.']*u.deg)  # Why do I have to invert PA?
        pixel_aperture = aperture.to_pixel(wcs)
        pixel_aperture.positions = [np.array(data.shape) / 2]  # dirty fix

        flux = pixel_aperture.do_photometry(data)[0][0]
        mean = flux / pixel_aperture.area
        bg, bg_err = find_bg(data)

        corrected_flux = flux - bg*pixel_aperture.area
        corrected_flux *= unit_conversion * np.cos(position.dec)
        flux_err = bg_err * pixel_aperture.area * unit_conversion * np.cos(position.dec)
        print(f'area = {pixel_aperture.area:.2f}, mean({mean:.3g}) - bg ({bg:.3g}) = {mean-bg:.3g} +- {bg_err:.3g}')
        print(f'flux = {corrected_flux:.3g} +- {flux_err:.3g} ({flux:.3g})')
        print('catalogue', galaxy['F_160'], galaxy['sigma_160'], corrected_flux/galaxy['F_160'])
        flux_Jy.append(corrected_flux)
        flux_err_Jy.append(flux_err)
        
        plt.figure()
        plt.title(f"HRS-{galaxy['HRS']} {band}: ${corrected_flux:.3g} \pm {flux_err:.3g}\ ({galaxy['F_160']:.3g} \pm {galaxy['sigma_160']:.3g})$ Jy")
        im = plt.imshow(data, interpolation='nearest', origin='lower', vmin=bg-bg_err, vmax=mean+bg_err, cmap='terrain')
        plt.contour(data, levels=[mean], colors=['k'])
        pixel_aperture.plot(color='w')
        cb = plt.colorbar(im)
        cb.ax.axhline(mean, c='w')
        cb.ax.axhline(mean, c='k')
        cb.ax.axhline(bg + bg_err, c='w', ls=':')
        cb.ax.axhline(bg, c='w', ls='--')
        cb.ax.axhline(bg - bg_err, c='w', ls=':')
        pdf.savefig()
        plt.close()

> Downloading HRS-1:
http://localhost:4000/api/cutout?radiusasec=96.0&pxsizeasec=2.0&radeg=154.41525&decdeg=22.80997222222222&hipsbaseuri=http://skies.esac.esa.int/Herschel/PACS160
0.020132677743670793 0.002300069042970157
0.02013225034504303 0.0015962231741387465
area = 1507.96, mean(0.0215) - bg (0.0201) = 0.00133 +- 0.0016
flux = 0.912 +- 1.09 (32.4)
catalogue 0.932 0.079 0.9781636032920453


> Downloading HRS-2:
http://localhost:4000/api/cutout?radiusasec=94.0&pxsizeasec=2.0&radeg=155.23804166666665&decdeg=25.364833333333333&hipsbaseuri=http://skies.esac.esa.int/Herschel/PACS160
0.018665887365723062 0.002065903189266343
0.018614324339165652 0.001461277969501305
area = 1661.12, mean(0.0227) - bg (0.0186) = 0.00406 +- 0.00146
flux = 3 +- 1.08 (37.7)
catalogue 2.808 0.179 1.0687790362191212
> Downloading HRS-3:
http://localhost:4000/api/cutout?radiusasec=78.0&pxsizeasec=2.0&radeg=155.86254166666663&decdeg=19.898527777777776&hipsbaseuri=http://skies.esac.esa.int/Herschel/PACS160
0.022365454384800077 0.00272625009748945
0.022136228926025542 0.0017889961838879545
area = 1194.59, mean(0.0222) - bg (0.0221) = 8.08e-05 +- 0.00179
flux = 0.0447 +- 0.99 (26.5)
catalogue 0.846 0.087 0.05283312883877898


> Downloading HRS-4:
http://localhost:4000/api/cutout?radiusasec=200.0&pxsizeasec=2.0&radeg=155.87741666666665&decdeg=19.865055555555557&hipsbaseuri=http://skies.esac.esa.int/Herschel/PACS160
0.02181892458139672 0.002572536999031441
0.021614508338388502 0.001772304936870584
area = 6597.34, mean(0.0284) - bg (0.0216) = 0.00681 +- 0.00177
flux = 20.8 +- 5.42 (188)
catalogue 22.675 1.165 0.9178142514587978
> Downloading HRS-5:
http://localhost:4000/api/cutout?radiusasec=162.0&pxsizeasec=2.0&radeg=156.6182083333333&decdeg=20.228194444444444&hipsbaseuri=http://skies.esac.esa.int/Herschel/PACS160
0.019301775503616468 0.003341268284676204
0.019245709566438375 0.00229900979487013
area = 3053.63, mean(0.0233) - bg (0.0192) = 0.00401 +- 0.0023
flux = 5.66 +- 3.24 (71)
catalogue 5.563 0.528 1.0180567567425878


> Downloading HRS-6:
http://localhost:4000/api/cutout?radiusasec=124.0&pxsizeasec=2.0&radeg=156.7548333333333&decdeg=28.639416666666666&hipsbaseuri=http://skies.esac.esa.int/Herschel/PACS160
0.022292428435353808 0.000981101258884877
0.022274669651237616 0.0006863795932521443
area = 1071.28, mean(0.0235) - bg (0.0223) = 0.00119 +- 0.000686
flux = 0.552 +- 0.318 (25.1)
catalogue 0.483 0.085 1.1430657786882608
> Downloading HRS-7:
http://localhost:4000/api/cutout?radiusasec=96.0&pxsizeasec=2.0&radeg=156.82662499999998&decdeg=28.50738888888889&hipsbaseuri=http://skies.esac.esa.int/Herschel/PACS160
0.02286965242582422 0.0027857320323266856
0.022841541158256044 0.002076394073869165
area = 1130.97, mean(0.0295) - bg (0.0228) = 0.00667 +- 0.00208
flux = 3.27 +- 1.02 (33.4)
catalogue 2.843 0.151 1.149113820025933


> Downloading HRS-8:
http://localhost:4000/api/cutout?radiusasec=420.0&pxsizeasec=2.0&radeg=157.33299999999997&decdeg=29.491444444444447&hipsbaseuri=http://skies.esac.esa.int/Herschel/PACS160
0.02289583691920761 0.0016108505959904033
0.022891152698972322 0.0010405264114447844
area = 10885.62, mean(0.0242) - bg (0.0229) = 0.00131 +- 0.00104
flux = 6.12 +- 4.86 (263)
catalogue 4.641 1.041 1.317665013468188
> Downloading HRS-9:
http://localhost:4000/api/cutout?radiusasec=164.0&pxsizeasec=2.0&radeg=158.23104166666664&decdeg=28.51172222222222&hipsbaseuri=http://skies.esac.esa.int/Herschel/PACS160
0.022111642654761825 0.0027535572716487246
0.022094640307461035 0.0019090094451077464
area = 4701.39, mean(0.0231) - bg (0.0221) = 0.000983 +- 0.00191
flux = 2 +- 3.88 (108)
catalogue 3.037 0.523 0.6586810305236698
> Downloading HRS-10:
http://localhost:4000/api/cutout?radiusasec=112.0&pxsizeasec=2.0&radeg=158.62424999999996&decdeg=35.25677777777778&hipsbaseuri=http://skies.esac.esa.int/Herschel/PA

In [6]:
n = len(flux_Jy)
with PdfPages(os.path.join('HRS', f'{band}_summary.pdf')) as pdf:
    plt.figure()
    plt.errorbar(HRS_catalogue['F_160'][:n], flux_Jy, flux_err_Jy, HRS_catalogue['sigma_160'][:n], fmt='none', alpha=.2)
    plt.grid(alpha=.2)
    x = [np.min(HRS_catalogue['F_160']), np.max(HRS_catalogue['F_160'])]
    plt.plot(x, x, 'k:')
    plt.xscale('log')
    plt.yscale('log')
    plt.ylim(x)
    plt.title(band)
    plt.ylabel('flux measured from HiPS map [Jy]')
    plt.xlabel('HRS flux [Jy]')
    pdf.savefig()
    plt.close()

    plt.figure()
    x = np.array(flux_Jy / HRS_catalogue['F_160'][:n])
    p16, p50, p84 = np.nanpercentile(x.data, [16, 50, 84])
    plt.hist(x, bins=np.logspace(-1, 1, 50), density=False, alpha=.5)
    plt.axvline(p50, c='k', ls='--', label=f'[{p16:.3g}, {p50:.3g}, {p84:.3g}]')
    plt.axvline(p16, c='k', ls=':')
    plt.axvline(p84, c='k', ls=':')
    plt.xscale('log')
    plt.grid(alpha=.2)
    plt.title(band)
    plt.xlabel('HiPS / HRS')
    plt.ylabel('number of galaxies')
    #plt.ylabel('probability')
    plt.legend()
    pdf.savefig()
    plt.close()
    
    plt.figure()
    x = np.array((flux_Jy - HRS_catalogue['F_160'][:n]) / flux_err_Jy)
    mu = np.nanmean(x)
    sigma = np.nanstd(x)
    p16, p50, p84 = np.nanpercentile(x.data, [16, 50, 84])
    plt.axvline(p16, c='b', ls=':')
    plt.axvline(p50, c='b', ls='--')
    plt.axvline(p84, c='b', ls=':')
    plt.hist(x, bins=np.linspace(-5, 5, 50), density=False, label=f'HiPS error: [{p16:.3g}, {p50:.3g}, {p84:.3g}]', alpha=.5)
    x = np.array((flux_Jy - HRS_catalogue['F_160'][:n]) / HRS_catalogue['sigma_160'][:n])
    mu = np.nanmean(x)
    sigma = np.nanstd(x)
    p16, p50, p84 = np.nanpercentile(x.data, [16, 50, 84])
    plt.axvline(p16, c='r', ls=':')
    plt.axvline(p50, c='r', ls='--')
    plt.axvline(p84, c='r', ls=':')
    plt.hist(x, bins=np.linspace(-5, 5, 50), density=False, label=f'HRS error: [{p16:.3g}, {p50:.3g}, {p84:.3g}]', alpha=.5)
    plt.grid(alpha=.2)
    plt.title(band)
    plt.xlabel('(HiPS - HRS) / error')
    plt.ylabel('number of galaxies')
    plt.legend()
    pdf.savefig()
    plt.close()

# -- OLD STUFF --

In [7]:
raise -1  # STOP HERE

TypeError: exceptions must derive from BaseException

In [ ]:
names = [f'HRS-{ID:03d}' for ID in HRS_catalogue['HRS']]

In [ ]:
positions = SkyCoord(HRS_catalogue['R.A.__J.2000_'], HRS_catalogue['Dec__J.2000_'],  unit=(u.hourangle, u.deg))

In [ ]:
catalogue = Table([names,
                   positions.ra.deg,
                   positions.dec.deg,
                   2 * HRS_catalogue['a'],
                   2.85 * np.ones(len(HRS_catalogue))
                  ], names=['ID', 'RA', 'DEC', 'RADIUS_ARCSEC', 'PIXEL_SIZE_ARCSEC'])

In [ ]:
catalogue

# 2. Explore available skymaps

Select the maps for this galaxy:

In [ ]:
importlib.reload(astrobrowser)
x = astrobrowser.DataExplorer(catalogue, 'ESAVO/P/HERSCHEL/PACS160')

In [ ]:
galaxy = HRS_catalogue[x.galaxy_index]
position = SkyCoord(galaxy['R.A.__J.2000_'], galaxy['Dec__J.2000_'],  unit=(u.hourangle, u.deg))
aperture = SkyEllipticalAperture(position, a=galaxy['a']*u.arcsec, b=galaxy['b']*u.arcsec, theta=-galaxy['P.A.']*u.deg)  # Why do I have to invert PA?
pixel_aperture = aperture.to_pixel(x.wcs)
pixel_aperture.positions = [np.array(x.data.shape) / 2]  # dirty fix
pixel_aperture.plot(x.ax1)

In [ ]:
flux = pixel_aperture.do_photometry(x.data)[0][0]
mean = flux / pixel_aperture.area
source_mask = np.nanmedian(x.data)
source_mask += np.sqrt(np.nanmean((x.data - source_mask)**2))
x.ax1.contour(x.data, levels=[source_mask], colors=['y'])

bg = np.nanmedian(x.data[x.data < source_mask])
#bg_err = np.sqrt(np.nanmean((bg - x.data[x.data < bg])**2))
bg_err = bg - np.nanmedian(x.data[x.data < bg])
x.ax1_cb.axhline(source_mask, c='y')
x.ax1_cb.axhline(bg+bg_err, c='k', ls=':')
x.ax1_cb.axhline(bg, c='k')
x.ax1_cb.axhline(bg-bg_err, c='k', ls=':')

#scale = (3.2/catalogue[x.galaxy_index]['PIXEL_SIZE_ARCSEC'])**2
scale = 1
corrected_flux = flux - bg*pixel_aperture.area
print(f'area = {pixel_aperture.area:.2f}, mean({mean:.3g}) - bg ({bg:.3g}) = {mean-bg:.3g} +- {bg_err:.3g}')
print(f'flux = {corrected_flux:.3g} +- {bg_err*pixel_aperture.area:.3g} ({flux:.3g})')
#print(f'masked = {np.sum(x.data[x.data > source_mask]) - bg*np.count_nonzero(x.data > source_mask):.3g} ({np.sum(x.data[x.data > source_mask]):.3g})')
#print(f'scaled = {corrected_flux*scale:.3g} +- {bg_err*(pixel_aperture.area)*scale:.3g}')
print('catalogue', galaxy['F_160'], galaxy['sigma_160'], corrected_flux*scale/galaxy['F_160'])

In [ ]:
pixel_aperture.area, pixel_aperture.area_overlap(x.data)

In [ ]:
aperture.positions, pixel_aperture.positions*2, x.data.shape

In [ ]:
galaxy

In [ ]:
x.header

In [ ]:
x.header['CDELT1']*3600